In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [3]:
# Read dataset
training = spark.read.csv('data3.csv', header=True, inferSchema=True)

In [4]:
training.show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
| Moises| 22|        10| 24893|
|   Luis| 18|         4| 12064|
|  Karla| 30|         2| 30548|
|  Maria| 55|        23| 32637|
|   Jose| 51|         8| 19511|
|    Luz| 35|         4| 38491|
|Enrique| 23|         4| 29000|
|Liliana| 54|        19| 34999|
|  Julia| 32|        10| 20000|
+-------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['Age', 'Experience'], outputCol='Independet Feature')

In [7]:
output = featureassembler.transform(training)

In [8]:
output.show()

+-------+---+----------+------+------------------+
|   Name|Age|Experience|Salary|Independet Feature|
+-------+---+----------+------+------------------+
| Moises| 22|        10| 24893|       [22.0,10.0]|
|   Luis| 18|         4| 12064|        [18.0,4.0]|
|  Karla| 30|         2| 30548|        [30.0,2.0]|
|  Maria| 55|        23| 32637|       [55.0,23.0]|
|   Jose| 51|         8| 19511|        [51.0,8.0]|
|    Luz| 35|         4| 38491|        [35.0,4.0]|
|Enrique| 23|         4| 29000|        [23.0,4.0]|
|Liliana| 54|        19| 34999|       [54.0,19.0]|
|  Julia| 32|        10| 20000|       [32.0,10.0]|
+-------+---+----------+------+------------------+



In [9]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independet Feature']

In [10]:
finalized_data = output.select('Independet Feature', 'Salary')

In [11]:
finalized_data.show()

+------------------+------+
|Independet Feature|Salary|
+------------------+------+
|       [22.0,10.0]| 24893|
|        [18.0,4.0]| 12064|
|        [30.0,2.0]| 30548|
|       [55.0,23.0]| 32637|
|        [51.0,8.0]| 19511|
|        [35.0,4.0]| 38491|
|        [23.0,4.0]| 29000|
|       [54.0,19.0]| 34999|
|       [32.0,10.0]| 20000|
+------------------+------+



In [12]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independet Feature', labelCol='Salary')
regressor=regressor.fit(train_data)

In [13]:
### Coefficients
regressor.coefficients

DenseVector([844.7144, -954.0694])

In [14]:
### Intercepts
regressor.intercept

8628.764150895666

In [15]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [16]:
pred_results.predictions.show()

+------------------+------+-----------------+
|Independet Feature|Salary|       prediction|
+------------------+------+-----------------+
|        [51.0,8.0]| 19511|44076.64582402617|
|       [54.0,19.0]| 34999|36116.02591327207|
+------------------+------+-----------------+



In [17]:
pred_results.meanAbsoluteError

12841.33586864912